# ZINBayes

In [1]:
from zinbayes.zinbayes import ZINBayes

In [4]:
import numpy as np

## Data set: Pollen et al, 2014
This data set contains 65 cells of four different types sequenced at low and high depth. We will use ZINBayes to visualize the cells in 2D with minimal technical bias.

We first visualize some properties of the data.

## Run ZINBayes without batch annotations

## Run ZINBayes with batch annotations

In [16]:
zinbayes = ZINBayes(n_components=2, n_mc_samples=1)
z = zinbayes.fit_transform(np.array([[10, 30, 49], [11, 20, 29], [1, 3, 5], [4, 0, 2]]))

Considering zero-inflation.
Considering cell-specific scalings.
100/100 [100%] ██████████████████████████████ Elapsed: 2s | Loss: 14.521


In [18]:
zinbayes

ZINBayes(X_test=None, batch_correction=False, minibatch_size=None,
     n_components=2, n_mc_samples=1, optimizer=None, scalings=True,
     test_iterations=100, validation=False, zero_inflation=True)